<a href="https://colab.research.google.com/github/25AlecZ/PredictingPlayerValue/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries + packages

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#install packages
!pip install --upgrade pip
!pip install pandas --upgrade pip
#!pip install scikit-learn --upgrade pip
!pip install xgboost --upgrade pip
!pip install sklearn --upgrade pip
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 29.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
bigframes 0.19.1 requires pandas<2.1.4,>=1.5.0, but you have panda

In [ ]:
#import modules
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
import shap
import gc

# Dataset Cleaning

In [ ]:
# @title
playerdf2018 = pd.read_csv('/content/drive/MyDrive/PlayerValue/playerdf2018.csv')
playerdf2018.drop('Unnamed: 0', axis=1, inplace=True)

playerdf2019 = pd.read_csv('/content/drive/MyDrive/PlayerValue/playerdf2019.csv')
playerdf2019.drop('Unnamed: 0', axis=1, inplace=True)

playerdf2020 = pd.read_csv('/content/drive/MyDrive/PlayerValue/playerdf2020.csv')
playerdf2020.drop('Unnamed: 0', axis=1, inplace=True)

playerdf2021 = pd.read_csv('/content/drive/MyDrive/PlayerValue/playerdf2021.csv')
playerdf2021.drop('Unnamed: 0', axis=1, inplace=True)

playerdf2022 = pd.read_csv('/content/drive/MyDrive/PlayerValue/playerdf2022.csv')
playerdf2022.drop('Unnamed: 0', axis=1, inplace=True)

playerdf2023 = pd.read_csv('/content/drive/MyDrive/PlayerValue/playerdf2023.csv')
playerdf2023.drop('Unnamed: 0', axis=1, inplace=True)

InjuryYearDurationUpdatedYearsdf = pd.read_csv('/content/drive/MyDrive/PlayerValue/InjuryYearDurationUpdatedYearsdf.csv')
InjuryYearDurationUpdatedYearsdf.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# @title
playerFrebdf = pd.concat([playerdf2018,playerdf2019, playerdf2020, playerdf2021, playerdf2022, playerdf2023], ignore_index = True)

In [ ]:
# @title
#concat all season dataframes from freb datasets
playerFrebdf['Age'] = pd.to_numeric(playerFrebdf['Age'])
playerFrebdf.rename(columns = {'player_url':'Player_url'}, inplace = True)
playerFrebdf = playerFrebdf.merge(InjuryYearDurationUpdatedYearsdf, on = 'Player_url', how = 'left')
playerFrebdf.count()
#no blank spaces in column names
#playerFrebdf.columns = playerFrebdf.columns.str.replace(' ','_')
#playerFrebdf['player_height_mtrs'] = pd.to_numeric(playerFrebdf['player_height_mtrs'])
#playerFrebdf.to_csv('/content/drive/MyDrive/PlayerValue/Frebdf.csv')

In [ ]:
# @title
#Injury with regular column names
Injury2018df = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2018df.csv')
Injury2018df.drop('Unnamed: 0', axis=1, inplace=True)

Injury2019df = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2019df.csv')
Injury2019df.drop('Unnamed: 0', axis=1, inplace=True)

Injury2020df = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2020df.csv')
Injury2020df.drop('Unnamed: 0', axis=1, inplace=True)

Injury2021df = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2021df.csv')
Injury2021df.drop('Unnamed: 0', axis=1, inplace=True)

Injury2022df = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2022df.csv')
Injury2022df.drop('Unnamed: 0', axis=1, inplace=True)

Injury2023df = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2023df.csv')
Injury2023df.drop('Unnamed: 0', axis=1, inplace=True)

#Injury with columns names including year in duration and type of injury
Injury2018YearDurationdf = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2018YearDurationdf.csv')
Injury2018YearDurationdf.drop('Unnamed: 0', axis=1, inplace=True)

Injury2019YearDurationdf = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2019YearDurationdf.csv')
Injury2019YearDurationdf.drop('Unnamed: 0', axis=1, inplace=True)

Injury2020YearDurationdf = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2020YearDurationdf.csv')
Injury2020YearDurationdf.drop('Unnamed: 0', axis=1, inplace=True)

Injury2021YearDurationdf = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2021YearDurationdf.csv')
Injury2021YearDurationdf.drop('Unnamed: 0', axis=1, inplace=True)

Injury2022YearDurationdf = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2022YearDurationdf.csv')
Injury2022YearDurationdf.drop('Unnamed: 0', axis=1, inplace=True)

Injury2023YearDurationdf = pd.read_csv('/content/drive/MyDrive/PlayerValue/Injury2023YearDurationdf.csv')
Injury2023YearDurationdf.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# @title
Injury2018df.rename(columns = {'Player_url':'player_url'}, inplace = True)
Injury2019df.rename(columns = {'Player_url':'player_url'}, inplace = True)
Injury2020df.rename(columns = {'Player_url':'player_url'}, inplace = True)
Injury2021df.rename(columns = {'Player_url':'player_url'}, inplace = True)
Injury2022df.rename(columns = {'Player_url':'player_url'}, inplace = True)
Injury2023df.rename(columns = {'Player_url':'player_url'}, inplace = True)

In [ ]:
# @title
#Injury Model
#Concatenate all data into overall injury df, then for loop search for player name, check player year duration and type duration, then replace NaN values with 0
InjuryYearDurationdf = pd.concat([Injury2018YearDurationdf, Injury2019YearDurationdf], ignore_index = True)
InjuryYearDurationdf = pd.concat([InjuryYearDurationdf, Injury2020YearDurationdf], ignore_index = True)
InjuryYearDurationdf = pd.concat([InjuryYearDurationdf, Injury2021YearDurationdf], ignore_index = True)
InjuryYearDurationdf = pd.concat([InjuryYearDurationdf, Injury2022YearDurationdf], ignore_index = True)
InjuryYearDurationdf = pd.concat([InjuryYearDurationdf, Injury2023YearDurationdf], ignore_index = True)

In [ ]:
# @title
for i in range(0, len(InjuryYearDurationdf)):
  playerUrl = InjuryYearDurationdf.loc[i, "Player_url"]
  for j in range(i, len(InjuryYearDurationdf)):
    if (InjuryYearDurationdf.loc[j, "Player_url"] == playerUrl):
        if (not InjuryYearDurationdf.isnull().loc[j, "Duration_2019"]):
          InjuryYearDurationdf.loc[i, "Duration_2019"] = InjuryYearDurationdf.loc[j, "Duration_2019"]
          InjuryYearDurationdf.loc[i, "Type_2019"] = InjuryYearDurationdf.loc[j, "Type_2019"]
        elif (not InjuryYearDurationdf.isnull().loc[j, "Duration_2020"]):
          InjuryYearDurationdf.loc[i, "Duration_2020"] = InjuryYearDurationdf.loc[j, "Duration_2020"]
          InjuryYearDurationdf.loc[i, "Type_2020"] = InjuryYearDurationdf.loc[j, "Type_2020"]
        elif (not InjuryYearDurationdf.isnull().loc[j, "Duration_2021"]):
          InjuryYearDurationdf.loc[i, "Duration_2021"] = InjuryYearDurationdf.loc[j, "Duration_2021"]
          InjuryYearDurationdf.loc[i, "Type_2021"] = InjuryYearDurationdf.loc[j, "Type_2021"]
        elif (not InjuryYearDurationdf.isnull().loc[j, "Duration_2022"]):
          InjuryYearDurationdf.loc[i, "Duration_2022"] = InjuryYearDurationdf.loc[j, "Duration_2022"]
          InjuryYearDurationdf.loc[i, "Type_2022"] = InjuryYearDurationdf.loc[j, "Type_2022"]
        elif (not InjuryYearDurationdf.isnull().loc[j, "Duration_2023"]):
          InjuryYearDurationdf.loc[i, "Duration_2023"] = InjuryYearDurationdf.loc[j, "Duration_2023"]
          InjuryYearDurationdf.loc[i, "Type_2023"] = InjuryYearDurationdf.loc[j, "Type_2023"]
  print(i)

In [ ]:
# @title
playerdf2018 = playerdf2018.merge(Injury2018df, on = 'player_url', how = 'left')
playerdf2019 = playerdf2019.merge(Injury2019df, on = 'player_url', how = 'left')
playerdf2020 = playerdf2020.merge(Injury2020df, on = 'player_url', how = 'left')
playerdf2021 = playerdf2021.merge(Injury2021df, on = 'player_url', how = 'left')
playerdf2022 = playerdf2022.merge(Injury2022df, on = 'player_url', how = 'left')
playerdf2023 = playerdf2023.merge(Injury2023df, on = 'player_url', how = 'left')

In [ ]:
# @title
#concat all season dataframes from freb datasets
playerFrebdf = pd.concat([playerdf2018,playerdf2019, playerdf2020, playerdf2021, playerdf2022, playerdf2023], ignore_index = True)
playerFrebdf['Age'] = pd.to_numeric(playerFrebdf['Age'])
playerFrebdf['player_height_mtrs'] = pd.to_numeric(playerFrebdf['player_height_mtrs'])
playerFrebdf.sample(2)

# Original Dataset and sliding windows

In [ ]:
# @title
InjuryYearDurationUpdatedYearsdf = InjuryYearDurationUpdatedYearsdf.drop_duplicates(subset=['Player_url'], keep = 'first')
InjuryYearDurationUpdatedYearsdf.count()
InjuryYearDurationUpdatedYearsdf.to_csv('/content/drive/MyDrive/PlayerValue/InjuryYearDurationUpdatedYearsdf.csv')

In [ ]:
# @title
#Clean dataframe so that each row only has one injury from that season year
for i in range (0, len(InjuryModeldf)):
  Year = InjuryModeldf.loc[i, 'Season_End_Year']
  if (Year == 2018):
    InjuryModeldf.at[i, 'Duration_2019'] = np.nan
    InjuryModeldf.at[i, 'Type_2019'] = np.nan
    InjuryModeldf.at[i, 'Duration_2020'] = np.nan
    InjuryModeldf.at[i, 'Type_2020'] = np.nan
    InjuryModeldf.at[i, 'Duration_2021'] = np.nan
    InjuryModeldf.at[i, 'Type_2021'] = np.nan
    InjuryModeldf.at[i, 'Duration_2022'] = np.nan
    InjuryModeldf.at[i, 'Type_2022'] = np.nan
    InjuryModeldf.at[i, 'Duration_2023'] = np.nan
    InjuryModeldf.at[i, 'Type_2023'] = np.nan
  elif (Year == 2019):
    InjuryModeldf.at[i, 'Duration_2018'] = np.nan
    InjuryModeldf.at[i, 'Type_2018'] = np.nan
    InjuryModeldf.at[i, 'Duration_2020'] = np.nan
    InjuryModeldf.at[i, 'Type_2020'] = np.nan
    InjuryModeldf.at[i, 'Duration_2021'] = np.nan
    InjuryModeldf.at[i, 'Type_2021'] = np.nan
    InjuryModeldf.at[i, 'Duration_2022'] = np.nan
    InjuryModeldf.at[i, 'Type_2022'] = np.nan
    InjuryModeldf.at[i, 'Duration_2023'] = np.nan
    InjuryModeldf.at[i, 'Type_2023'] = np.nan
  elif (Year == 2020):
    InjuryModeldf.at[i, 'Duration_2018'] = np.nan
    InjuryModeldf.at[i, 'Type_2018'] = np.nan
    InjuryModeldf.at[i, 'Duration_2019'] = np.nan
    InjuryModeldf.at[i, 'Type_2019'] = np.nan
    InjuryModeldf.at[i, 'Duration_2021'] = np.nan
    InjuryModeldf.at[i, 'Type_2021'] = np.nan
    InjuryModeldf.at[i, 'Duration_2022'] = np.nan
    InjuryModeldf.at[i, 'Type_2022'] = np.nan
    InjuryModeldf.at[i, 'Duration_2023'] = np.nan
    InjuryModeldf.at[i, 'Type_2023'] = np.nan
  elif (Year == 2021):
    InjuryModeldf.at[i, 'Duration_2018'] = np.nan
    InjuryModeldf.at[i, 'Type_2018'] = np.nan
    InjuryModeldf.at[i, 'Duration_2019'] = np.nan
    InjuryModeldf.at[i, 'Type_2019'] = np.nan
    InjuryModeldf.at[i, 'Duration_2020'] = np.nan
    InjuryModeldf.at[i, 'Type_2020'] = np.nan
    InjuryModeldf.at[i, 'Duration_2022'] = np.nan
    InjuryModeldf.at[i, 'Type_2022'] = np.nan
    InjuryModeldf.at[i, 'Duration_2023'] = np.nan
    InjuryModeldf.at[i, 'Type_2023'] = np.nan
  elif (Year == 2022):
    InjuryModeldf.at[i, 'Duration_2018'] = np.nan
    InjuryModeldf.at[i, 'Type_2018'] = np.nan
    InjuryModeldf.at[i, 'Duration_2019'] = np.nan
    InjuryModeldf.at[i, 'Type_2019'] = np.nan
    InjuryModeldf.at[i, 'Duration_2020'] = np.nan
    InjuryModeldf.at[i, 'Type_2020'] = np.nan
    InjuryModeldf.at[i, 'Duration_2021'] = np.nan
    InjuryModeldf.at[i, 'Type_2021'] = np.nan
    InjuryModeldf.at[i, 'Duration_2023'] = np.nan
    InjuryModeldf.at[i, 'Type_2023'] = np.nan
  elif (Year == 2023):
    InjuryModeldf.at[i, 'Duration_2018'] = np.nan
    InjuryModeldf.at[i, 'Type_2018'] = np.nan
    InjuryModeldf.at[i, 'Duration_2019'] = np.nan
    InjuryModeldf.at[i, 'Type_2019'] = np.nan
    InjuryModeldf.at[i, 'Duration_2020'] = np.nan
    InjuryModeldf.at[i, 'Type_2020'] = np.nan
    InjuryModeldf.at[i, 'Duration_2021'] = np.nan
    InjuryModeldf.at[i, 'Type_2021'] = np.nan
    InjuryModeldf.at[i, 'Duration_2022'] = np.nan
    InjuryModeldf.at[i, 'Type_2022'] = np.nan

#InjuryModeldf.to_csv('/content/drive/MyDrive/PlayerValue/FrebInjuryPerYear.csv')

In [ ]:
InjuryModeldf = pd.read_csv('/content/drive/MyDrive/PlayerValue/FrebInjuryPerYear.csv')
InjuryModeldf.drop('Unnamed: 0', axis=1, inplace=True)

Columns (233,242,245,247,249,251,253) have mixed types. Specify dtype option on import or set low_memory=False.


In [ ]:
Testdf = pd.read_csv('/content/drive/MyDrive/PlayerValue/Testdf.csv')
Testdf.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
Testdf.to_csv('/content/drive/MyDrive/PlayerValue/Testdfmodified.csv')

In [ ]:
Testdf = pd.read_csv('/content/drive/MyDrive/PlayerValue/Testdfmodified.csv')
Testdf.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
# @title
Testdf = InjuryModeldf.copy()
for i in range (0, len(InjuryModeldf)):
  print(i)
  if (not (InjuryModeldf.loc[i, 'Player_x'] == 'John Stones' or InjuryModeldf.loc[i, 'Player_x'] == 'Danny Welbeck')):
    Testdf.drop(i, inplace = True)

In [ ]:
#make sure to include type
#parameter years needed, concatenate dataframe and add injury duration + ground truth
def slidingWindowstest(years):
  InjuryDurationdf = pd.DataFrame()
  #truth dataframe for only duration of injuries across years
  InjuryDurationdf['duration_truth'] = np.nan
  #dataframe for stats across all years
  Statsdf = Testdf.copy()

  for year in range(1, years):
    for col in Testdf.columns:
      #duplicates columns for years-1
      Statsdf[str(col) + 'year' + str(year+1)] = np.nan

  Statsdf['duration'] = 0
  Statsdf['injuryType'] = ''



#concatenating dataframe
  for originalRow in range (0, len(Statsdf)):
    print('OriginalRow: ' + str(originalRow))
    url = Statsdf.loc[originalRow, 'Url']
    originalSeason = Statsdf.loc[originalRow, 'Season_End_Year'] #2018
    truthSeason = originalSeason + years #2019
    foundtruth = False
    #duration not working properly for year one, truth duration also doesn't work
    if (not str(Statsdf.loc[originalRow, 'Type_' + str(originalSeason)]) == 'nan'):
      Statsdf.loc[originalRow, 'injuryType'] += str(Statsdf.loc[originalRow, 'Type_' + str(originalSeason)])
    if (not str(Statsdf.loc[originalRow, 'Duration_' + str(originalSeason)]) == 'nan'):
      Statsdf.loc[originalRow, 'duration'] += Statsdf.loc[originalRow, 'Duration_' + str(originalSeason)]

    if (truthSeason > 2023):
      break
    for changedRow in range (originalRow, len(Statsdf)):
      currentSeason = Statsdf.loc[changedRow, 'Season_End_Year'] #2019
      if (currentSeason > truthSeason):
        break
      #if url matches
      if (Statsdf.loc[changedRow, 'Url'] == url and currentSeason > originalSeason):
        #year is not truth
        if (currentSeason < truthSeason):
          #iterate through all columns to concatenate to original row
          for col in range(0, len(Testdf.columns)):
            Statsdf.iloc[originalRow, (col + (currentSeason - originalSeason) * len(Testdf.columns))] = Statsdf.iloc[changedRow, col]
          if ((not str(Statsdf.loc[changedRow, 'Type_' + str(currentSeason)]) == 'nan') and years != 1):
            Statsdf.loc[originalRow, 'injuryType'] += str(Statsdf.loc[changedRow, 'Type_' + str(currentSeason)]) #should it be currentSeason?
          if ((not str(Statsdf.loc[changedRow, 'Duration_' + str(currentSeason)]) == 'nan') and years != 1):
            Statsdf.loc[originalRow, 'duration'] += Statsdf.loc[changedRow, 'Duration_' + str(currentSeason)]

        #if year is truth, add duration of injury
        elif (currentSeason == truthSeason):
          foundtruth = True
          if (str(Statsdf.loc[changedRow, 'Duration_' + str(truthSeason)]) == 'nan'):
            InjuryDurationdf.loc[len(InjuryDurationdf), 'duration_truth'] = 0
          else:
            InjuryDurationdf.loc[len(InjuryDurationdf), 'duration_truth'] = Statsdf.loc[changedRow, 'Duration_' + str(truthSeason)]
          break;

    if (foundtruth == False):
      InjuryDurationdf.loc[len(InjuryDurationdf), 'duration_truth'] = np.nan

#error may be how I am joining df

  Statsdf = Statsdf.join(InjuryDurationdf)

  #return modified df
  return Statsdf

In [ ]:
#make sure to include type
#parameter years needed, concatenate dataframe and add injury duration + ground truth
def slidingWindows(years):
  InjuryDurationdf = pd.DataFrame()
  #truth dataframe for only duration of injuries across years
  InjuryDurationdf['duration_truth'] = np.nan
  #dataframe for stats across all years
  Statsdf = InjuryModeldf.copy()

  for year in range(1, years):
    for col in InjuryModeldf.columns:
      #duplicates columns for years-1
      Statsdf[str(col) + 'year' + str(year+1)] = np.nan

  Statsdf['duration'] = 0
  Statsdf['injuryType'] = ''



#concatenating dataframe
  for originalRow in range (0, len(Statsdf)):
    print('OriginalRow: ' + str(originalRow))
    url = Statsdf.loc[originalRow, 'Url']
    originalSeason = Statsdf.loc[originalRow, 'Season_End_Year'] #2018
    truthSeason = originalSeason + years #2019
    foundtruth = False
    #duration not working properly for year one, truth duration also doesn't work
    if (not str(Statsdf.loc[originalRow, 'Type_' + str(originalSeason)]) == 'nan'):
      Statsdf.loc[originalRow, 'injuryType'] += str(Statsdf.loc[originalRow, 'Type_' + str(originalSeason)])
    if (not str(Statsdf.loc[originalRow, 'Duration_' + str(originalSeason)]) == 'nan'):
      Statsdf.loc[originalRow, 'duration'] += Statsdf.loc[originalRow, 'Duration_' + str(originalSeason)]

    if (truthSeason > 2023):
      break
    for changedRow in range (originalRow + 2800, len(Statsdf)):
      currentSeason = Statsdf.loc[changedRow, 'Season_End_Year'] #2019
      if (currentSeason > truthSeason):
        break
      #if url matches
      if (Statsdf.loc[changedRow, 'Url'] == url and currentSeason > originalSeason):
        #year is not truth
        if (currentSeason < truthSeason):
          #iterate through all columns to concatenate to original row
          for col in range(0, len(Testdf.columns)):
            Statsdf.iloc[originalRow, (col + (currentSeason - originalSeason) * len(Testdf.columns))] = Statsdf.iloc[changedRow, col]
          if ((not str(Statsdf.loc[changedRow, 'Type_' + str(currentSeason)]) == 'nan') and years != 1):
            Statsdf.loc[originalRow, 'injuryType'] += str(Statsdf.loc[changedRow, 'Type_' + str(currentSeason)]) #should it be currentSeason?
          if ((not str(Statsdf.loc[changedRow, 'Duration_' + str(currentSeason)]) == 'nan') and years != 1):
            Statsdf.loc[originalRow, 'duration'] += Statsdf.loc[changedRow, 'Duration_' + str(currentSeason)]

        #if year is truth, add duration of injury
        elif (currentSeason == truthSeason):
          foundtruth = True
          if (str(Statsdf.loc[changedRow, 'Duration_' + str(truthSeason)]) == 'nan'):
            InjuryDurationdf.loc[len(InjuryDurationdf), 'duration_truth'] = 0
          else:
            InjuryDurationdf.loc[len(InjuryDurationdf), 'duration_truth'] = Statsdf.loc[changedRow, 'Duration_' + str(truthSeason)]
          break;

    if (foundtruth == False):
      InjuryDurationdf.loc[len(InjuryDurationdf), 'duration_truth'] = np.nan

#error may be how I am joining df

  Statsdf = Statsdf.join(InjuryDurationdf)

  #return modified df
  return Statsdf

In [ ]:
InjuryModel5year = slidingWindows(5)

In [ ]:
InjuryModel5year.to_csv('/content/drive/MyDrive/PlayerValue/InjuryModel5year.csv')

# Read concatenated csv


In [ ]:
#read df for each year model
InjuryModelOneYeardf = pd.read_csv('/content/drive/MyDrive/PlayerValue/InjuryModel1year.csv')
InjuryModelOneYeardf.drop('Unnamed: 0', axis=1, inplace=True)
InjuryModelOneYeardf.drop(columns = ['Season_End_Year', 'season_start_year', 'Season', 'contract_expiry', 'player_num', 'player_name', 'Player_y', 'Player_x', 'comp_name', 'Url', 'Player_url', 'player_dob', 'player_nationality', 'current_club', 'squad','player_position', 'contract_expiry', 'date_joined', 'joined_from', 'player_market_value_euro', 'region','Duration_2018', 'Duration_2019', 'Duration_2020', 'Duration_2021', 'Duration_2022', 'Duration_2023', 'Type_2018', 'Type_2019','Type_2020', 'Type_2021', 'Type_2022', 'Type_2023'], inplace = True)
x1year = InjuryModelOneYeardf.dropna(subset=['duration_truth'])
y1year = x1year['duration_truth'].copy()

Columns (233,242,245,247,249,251,253,255) have mixed types. Specify dtype option on import or set low_memory=False.


In [ ]:
InjuryModelTwoYeardf = pd.read_csv('/content/drive/MyDrive/PlayerValue/InjuryModel2year.csv')
InjuryModelTwoYeardf.drop('Unnamed: 0', axis=1, inplace=True)
InjuryModelTwoYeardf.drop(columns = ['Season_End_Year', 'season_start_year', 'Season', 'contract_expiry', 'player_num', 'player_name', 'Player_y', 'Player_x', 'comp_name', 'Url', 'Player_url', 'player_dob', 'player_nationality', 'current_club', 'squad','player_position', 'contract_expiry', 'date_joined', 'joined_from', 'player_market_value_euro', 'region','Duration_2018', 'Duration_2019', 'Duration_2020', 'Duration_2021', 'Duration_2022', 'Duration_2023', 'Type_2018', 'Type_2019','Type_2020', 'Type_2021', 'Type_2022', 'Type_2023',
                                     'Season_End_Yearyear2', 'season_start_yearyear2', 'Seasonyear2', 'contract_expiryyear2', 'player_numyear2', 'player_nameyear2', 'Player_yyear2', 'Player_xyear2', 'comp_nameyear2', 'Urlyear2', 'Player_urlyear2', 'player_dobyear2', 'player_nationalityyear2', 'current_clubyear2', 'squadyear2','player_positionyear2', 'contract_expiryyear2', 'date_joinedyear2', 'joined_fromyear2', 'player_market_value_euroyear2', 'regionyear2','Duration_2018year2', 'Duration_2019year2', 'Duration_2020year2', 'Duration_2021year2', 'Duration_2022year2', 'Duration_2023year2', 'Type_2018year2', 'Type_2019year2','Type_2020year2', 'Type_2021year2', 'Type_2022year2', 'Type_2023year2'], inplace = True)
x2year = InjuryModelTwoYeardf.dropna(subset=['duration_truth'])
y2year = x2year['duration_truth'].copy()

Columns (233,242,245,247,249,251,253,255,256,257,258,259,279,474,475,476,477,479,480,481,482,483,485,486,488,489,490,491,493,498,500,502,504,508) have mixed types. Specify dtype option on import or set low_memory=False.


In [ ]:
InjuryModelThreeYeardf = pd.read_csv('/content/drive/MyDrive/PlayerValue/InjuryModel3year.csv')
InjuryModelThreeYeardf.drop('Unnamed: 0', axis=1, inplace=True)
InjuryModelThreeYeardf.drop(columns = ['Season_End_Year', 'season_start_year', 'Season', 'contract_expiry', 'player_num', 'player_name', 'Player_y', 'Player_x', 'comp_name', 'Url', 'Player_url', 'player_dob', 'player_nationality', 'current_club', 'squad','player_position', 'contract_expiry', 'date_joined', 'joined_from', 'player_market_value_euro', 'region','Duration_2018', 'Duration_2019', 'Duration_2020', 'Duration_2021', 'Duration_2022', 'Duration_2023', 'Type_2018', 'Type_2019','Type_2020', 'Type_2021', 'Type_2022', 'Type_2023',
                                       'Season_End_Yearyear2', 'season_start_yearyear2', 'Seasonyear2', 'contract_expiryyear2', 'player_numyear2', 'player_nameyear2', 'Player_yyear2', 'Player_xyear2', 'comp_nameyear2', 'Urlyear2', 'Player_urlyear2', 'player_dobyear2', 'player_nationalityyear2', 'current_clubyear2', 'squadyear2','player_positionyear2', 'contract_expiryyear2', 'date_joinedyear2', 'joined_fromyear2', 'player_market_value_euroyear2', 'regionyear2','Duration_2018year2', 'Duration_2019year2', 'Duration_2020year2', 'Duration_2021year2', 'Duration_2022year2', 'Duration_2023year2', 'Type_2018year2', 'Type_2019year2','Type_2020year2', 'Type_2021year2', 'Type_2022year2', 'Type_2023year2',
                                       'Season_End_Yearyear3', 'season_start_yearyear3', 'Seasonyear3', 'contract_expiryyear3', 'player_numyear3', 'player_nameyear3', 'Player_yyear3', 'Player_xyear3', 'comp_nameyear3', 'Urlyear3', 'Player_urlyear3', 'player_dobyear3', 'player_nationalityyear3', 'current_clubyear3', 'squadyear3','player_positionyear3', 'contract_expiryyear3', 'date_joinedyear3', 'joined_fromyear3', 'player_market_value_euroyear3', 'regionyear3','Duration_2018year3', 'Duration_2019year3', 'Duration_2020year3', 'Duration_2021year3', 'Duration_2022year3', 'Duration_2023year3', 'Type_2018year3', 'Type_2019year3','Type_2020year3', 'Type_2021year3', 'Type_2022year3', 'Type_2023year3'], inplace = True)
x3year = InjuryModelThreeYeardf.dropna(subset=['duration_truth'])
y3year = x3year['duration_truth'].copy()

Columns (233,242,245,247,249,251,253,255,256,257,258,259,279,474,475,476,477,479,480,481,482,483,485,486,488,489,490,491,493,498,500,502,508,509,510,511,512,532,727,728,729,730,732,733,734,735,736,738,739,741,742,743,744,746,753,755,757,761) have mixed types. Specify dtype option on import or set low_memory=False.


In [ ]:
InjuryModelFourYeardf = pd.read_csv('/content/drive/MyDrive/PlayerValue/InjuryModel4year.csv')
InjuryModelFourYeardf.drop('Unnamed: 0', axis=1, inplace=True)
InjuryModelFourYeardf.drop(columns = ['Season_End_Year', 'season_start_year', 'Season', 'contract_expiry', 'player_num', 'player_name', 'Player_y', 'Player_x', 'comp_name', 'Url', 'Player_url', 'player_dob', 'player_nationality', 'current_club', 'squad','player_position', 'contract_expiry', 'date_joined', 'joined_from', 'player_market_value_euro', 'region','Duration_2018', 'Duration_2019', 'Duration_2020', 'Duration_2021', 'Duration_2022', 'Duration_2023', 'Type_2018', 'Type_2019','Type_2020', 'Type_2021', 'Type_2022', 'Type_2023',
                                     'Season_End_Yearyear2', 'season_start_yearyear2', 'Seasonyear2', 'contract_expiryyear2', 'player_numyear2', 'player_nameyear2', 'Player_yyear2', 'Player_xyear2', 'comp_nameyear2', 'Urlyear2', 'Player_urlyear2', 'player_dobyear2', 'player_nationalityyear2', 'current_clubyear2', 'squadyear2','player_positionyear2', 'contract_expiryyear2', 'date_joinedyear2', 'joined_fromyear2', 'player_market_value_euroyear2', 'regionyear2','Duration_2018year2', 'Duration_2019year2', 'Duration_2020year2', 'Duration_2021year2', 'Duration_2022year2', 'Duration_2023year2', 'Type_2018year2', 'Type_2019year2','Type_2020year2', 'Type_2021year2', 'Type_2022year2', 'Type_2023year2',
                                     'Season_End_Yearyear3', 'season_start_yearyear3', 'Seasonyear3', 'contract_expiryyear3', 'player_numyear3', 'player_nameyear3', 'Player_yyear3', 'Player_xyear3', 'comp_nameyear3', 'Urlyear3', 'Player_urlyear3', 'player_dobyear3', 'player_nationalityyear3', 'current_clubyear3', 'squadyear3','player_positionyear3', 'contract_expiryyear3', 'date_joinedyear3', 'joined_fromyear3', 'player_market_value_euroyear3', 'regionyear3','Duration_2018year3', 'Duration_2019year3', 'Duration_2020year3', 'Duration_2021year3', 'Duration_2022year3', 'Duration_2023year3', 'Type_2018year3', 'Type_2019year3','Type_2020year3', 'Type_2021year3', 'Type_2022year3', 'Type_2023year3',
                                     'Season_End_Yearyear4', 'season_start_yearyear4', 'Seasonyear4', 'contract_expiryyear4', 'player_numyear4', 'player_nameyear4', 'Player_yyear4', 'Player_xyear4', 'comp_nameyear4', 'Urlyear4', 'Player_urlyear4', 'player_dobyear4', 'player_nationalityyear4', 'current_clubyear4', 'squadyear4','player_positionyear4', 'contract_expiryyear4', 'date_joinedyear4', 'joined_fromyear4', 'player_market_value_euroyear4', 'regionyear4','Duration_2018year4', 'Duration_2019year4', 'Duration_2020year4', 'Duration_2021year4', 'Duration_2022year4', 'Duration_2023year4', 'Type_2018year4', 'Type_2019year4','Type_2020year4', 'Type_2021year4', 'Type_2022year4', 'Type_2023year4'], inplace = True)
x4year = InjuryModelFourYeardf.dropna(subset=['duration_truth'])
y4year = x4year['duration_truth'].copy()

Columns (233,242,245,247,249,251,253,255,256,257,258,259,279,474,475,476,477,479,480,481,482,483,485,486,488,489,490,491,493,498,500,508,509,510,511,512,532,727,728,729,730,732,733,734,735,736,738,739,741,742,743,744,746,753,755,761,762,763,764,765,785,980,981,982,983,985,986,987,988,989,991,992,994,995,996,997,999,1008,1010,1014) have mixed types. Specify dtype option on import or set low_memory=False.


In [ ]:
InjuryModelFiveYeardf = pd.read_csv('/content/drive/MyDrive/PlayerValue/InjuryModel5year.csv')
InjuryModelFiveYeardf.drop('Unnamed: 0', axis=1, inplace=True)
InjuryModelFiveYeardf.drop(columns = ['Season_End_Year', 'season_start_year', 'Season', 'contract_expiry', 'player_num', 'player_name', 'Player_y', 'Player_x', 'comp_name', 'Url', 'Player_url', 'player_dob', 'player_nationality', 'current_club', 'squad','player_position', 'contract_expiry', 'date_joined', 'joined_from', 'player_market_value_euro', 'region','Duration_2018', 'Duration_2019', 'Duration_2020', 'Duration_2021', 'Duration_2022', 'Duration_2023', 'Type_2018', 'Type_2019','Type_2020', 'Type_2021', 'Type_2022', 'Type_2023',
                                     'Season_End_Yearyear2', 'season_start_yearyear2', 'Seasonyear2', 'contract_expiryyear2', 'player_numyear2', 'player_nameyear2', 'Player_yyear2', 'Player_xyear2', 'comp_nameyear2', 'Urlyear2', 'Player_urlyear2', 'player_dobyear2', 'player_nationalityyear2', 'current_clubyear2', 'squadyear2','player_positionyear2', 'contract_expiryyear2', 'date_joinedyear2', 'joined_fromyear2', 'player_market_value_euroyear2', 'regionyear2','Duration_2018year2', 'Duration_2019year2', 'Duration_2020year2', 'Duration_2021year2', 'Duration_2022year2', 'Duration_2023year2', 'Type_2018year2', 'Type_2019year2','Type_2020year2', 'Type_2021year2', 'Type_2022year2', 'Type_2023year2',
                                     'Season_End_Yearyear3', 'season_start_yearyear3', 'Seasonyear3', 'contract_expiryyear3', 'player_numyear3', 'player_nameyear3', 'Player_yyear3', 'Player_xyear3', 'comp_nameyear3', 'Urlyear3', 'Player_urlyear3', 'player_dobyear3', 'player_nationalityyear3', 'current_clubyear3', 'squadyear3','player_positionyear3', 'contract_expiryyear3', 'date_joinedyear3', 'joined_fromyear3', 'player_market_value_euroyear3', 'regionyear3','Duration_2018year3', 'Duration_2019year3', 'Duration_2020year3', 'Duration_2021year3', 'Duration_2022year3', 'Duration_2023year3', 'Type_2018year3', 'Type_2019year3','Type_2020year3', 'Type_2021year3', 'Type_2022year3', 'Type_2023year3',
                                     'Season_End_Yearyear4', 'season_start_yearyear4', 'Seasonyear4', 'contract_expiryyear4', 'player_numyear4', 'player_nameyear4', 'Player_yyear4', 'Player_xyear4', 'comp_nameyear4', 'Urlyear4', 'Player_urlyear4', 'player_dobyear4', 'player_nationalityyear4', 'current_clubyear4', 'squadyear4','player_positionyear4', 'contract_expiryyear4', 'date_joinedyear4', 'joined_fromyear4', 'player_market_value_euroyear4', 'regionyear4','Duration_2018year4', 'Duration_2019year4', 'Duration_2020year4', 'Duration_2021year4', 'Duration_2022year4', 'Duration_2023year4', 'Type_2018year4', 'Type_2019year4','Type_2020year4', 'Type_2021year4', 'Type_2022year4', 'Type_2023year4',
                                     'Season_End_Yearyear5', 'season_start_yearyear5', 'Seasonyear5', 'contract_expiryyear5', 'player_numyear5', 'player_nameyear5', 'Player_yyear5', 'Player_xyear5', 'comp_nameyear5', 'Urlyear5', 'Player_urlyear5', 'player_dobyear5', 'player_nationalityyear5', 'current_clubyear5', 'squadyear5','player_positionyear5', 'contract_expiryyear5', 'date_joinedyear5', 'joined_fromyear5', 'player_market_value_euroyear5', 'regionyear5','Duration_2018year5', 'Duration_2019year5', 'Duration_2020year5', 'Duration_2021year5', 'Duration_2022year5', 'Duration_2023year5', 'Type_2018year5', 'Type_2019year5','Type_2020year5', 'Type_2021year5', 'Type_2022year5', 'Type_2023year5',], inplace = True)
x5year = InjuryModelFiveYeardf.dropna(subset=['duration_truth'])
y5year = x5year['duration_truth'].copy()

Columns (233,242,245,247,249,251,253,255,256,257,258,259,279,474,475,476,477,479,480,481,482,483,485,486,488,489,490,491,493,498,508,509,510,511,512,532,727,728,729,730,732,733,734,735,736,738,739,741,742,743,744,746,753,761,762,763,764,765,785,980,981,982,983,985,986,987,988,989,991,992,994,995,996,997,999,1008,1014,1015,1016,1017,1018,1038,1233,1234,1235,1236,1238,1239,1240,1241,1242,1244,1245,1247,1248,1249,1250,1252,1263,1267) have mixed types. Specify dtype option on import or set low_memory=False.


# OneHotEncoding + SHAP for Injuries

In [ ]:
types = x1year.dtypes

In [ ]:
for i in range(0, len(types)):
  print(types[i])

In [ ]:
# @title
counter = 0
for col in x1year.columns:
  column_name = str(col)
  if (x1year.column_name.dtype != "object"):
    x1year.drop(counter, axis = 1)
    counter = counter + 1

## One Year

In [ ]:
#One year model
x1yearEncoding = pd.DataFrame()
x1year["Squad"] = x1year["Squad"].astype("category")
extracted = x1year["Squad"]
x1year.drop("Squad", axis = 1)
x1yearEncoding = pd.concat([x1yearEncoding, extracted], axis=1)
x1year["Comp"] = x1year["Comp"].astype("category")
extracted = x1year["Comp"]
x1year.drop("Comp", axis = 1)
x1yearEncoding = pd.concat([x1yearEncoding, extracted], axis=1)
x1year["Pos"] = x1year["Pos"].astype("category")
extracted = x1year["Pos"]
x1year.drop("Pos", axis = 1)
x1yearEncoding = pd.concat([x1yearEncoding, extracted], axis=1)
x1year["Nation"] = x1year["Nation"].astype("category")
extracted = x1year["Nation"]
x1year.drop("Nation", axis = 1)
x1yearEncoding = pd.concat([x1yearEncoding, extracted], axis=1)
x1year["player_foot"] = x1year["player_foot"].astype("category")
extracted = x1year["player_foot"]
x1year.drop("player_foot", axis = 1)
x1yearEncoding = pd.concat([x1yearEncoding, extracted], axis=1)
x1year["country"] = x1year["country"].astype("category")
extracted = x1year["country"]
x1year.drop("country", axis = 1)
x1yearEncoding = pd.concat([x1yearEncoding, extracted], axis=1)
x1year["injuryType"] = x1year["injuryType"].astype("category")
extracted = x1year["injuryType"]
x1year.drop("injuryType", axis = 1)
x1yearEncoding = pd.concat([x1yearEncoding, extracted], axis=1)
x1year.drop(['duration_truth'], axis = 1, inplace = True)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [ ]:
#One hot encoder also converting floats+duration to booleans

enc = OneHotEncoder()
enc.fit(x1yearEncoding)

#transform categorical features
X1year_encoded = enc.transform(x1yearEncoding).toarray()
feature_names = x1yearEncoding.columns
new_feature_names = enc.get_feature_names_out(feature_names)
Xencoded = pd.DataFrame(X1year_encoded, columns= new_feature_names)

In [ ]:
X = x1year.join(Xencoded)

In [ ]:
X.shape

(8987, 3144)

In [ ]:
X = x1year

In [ ]:
x1year_train, x1year_test, y1year_train, y1year_test = train_test_split(X, y1year, random_state=1, train_size = 0.8)
InjuryModelXGB1year = XGBRegressor(tree_method="approx", seed=1, enable_categorical = True)
InjuryModelXGB1year.fit(x1year_train, y1year_train, verbose = True, early_stopping_rounds = 10, eval_set = [(x1year_test, y1year_test)])

[0]	validation_0-rmse:87.10556
[1]	validation_0-rmse:87.28685
[2]	validation_0-rmse:87.63194
[3]	validation_0-rmse:87.98877


`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.


[4]	validation_0-rmse:88.20354
[5]	validation_0-rmse:88.66903
[6]	validation_0-rmse:88.85886
[7]	validation_0-rmse:89.26891
[8]	validation_0-rmse:89.28460
[9]	validation_0-rmse:89.33695
[10]	validation_0-rmse:89.29079


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
InjuryModelXGB1year.save_model('InjuryModelXGB1year.json')
#files.download('InjuryModelXGB1year.json')

In [ ]:
InjuryModelXGB1year.load_model('InjuryModelXGB1year.json')

In [ ]:
#y1year_pred = InjuryModelXGB1year.predict(x1year_test)
shap_values = InjuryModelXGB1year.predict(XGBRegressor(x1year_test, enable_categorical=True), pred_contribs=True)
#print(r2_score(y1year_test, y1year_pred))
#print(mean_absolute_error(y1year_test, y1year_pred))
#print(mean_squared_error(y1year_test, y1year_pred))

Pass `objective` as keyword args.


TypeError: XGBModel.predict() got an unexpected keyword argument 'pred_contribs'

In [ ]:
explainer = shap.TreeExplainer(InjuryModelXGB1year)
shap_values = explainer(X)

In [ ]:
shap.summary_plot(shap_values, X, plot_type='bar')

In [ ]:
# waterfall plot for first observation
shap.plots.beeswarm(shap_values, max_display = 20)

In [ ]:
print(shap_values[0])

In [ ]:
print(shap_values.base_values.shape)

# XGB Injuries model

## One Year

In [ ]:
#Iterating thorugh hyperparameters of model
x1year_train, x1year_test, y1year_train, y1year_test = train_test_split(x1year, y1year, random_state=1, train_size = 0.8)
for depth in [None, 5, 10, 15, 20]:
  for learning_rate in [0.1, 0.01, 0.001]:
    for min_child_weight in [1,2,3]:
      for gamma in [0.1, 0.2, 0.3]:
        InjuryModelXGB1year = xgb.XGBRegressor(tree_method="approx", seed=1, max_depth = depth, learning_rate = learning_rate, min_child_weight = min_child_weight, gamma = gamma, enable_categorical = True)
        InjuryModelXGB1year.fit(x1year_train, y1year_train, verbose = True, early_stopping_rounds = 10, eval_set = [(x1year_test, y1year_test)])
        y1year_pred = InjuryModelXGB1year.predict(x1year_test)
        print(r2_score(y1year_test, y1year_pred))
        InjuryModelXGB1year.save_model('xgb_injury_1year_depth' + str(depth) + '_learningrate' + str(learning_rate) + '_childweight' + str(min_child_weight) + '_gamma' + str(gamma) + '_r2score' + str(r2_score(y1year_test, y1year_pred)) + '.json')
        files.download('xgb_injury_1year_depth' + str(depth) + '_learningrate' + str(learning_rate) + '_childweight' + str(min_child_weight) + '_gamma' + str(gamma) + '_r2score' + str(r2_score(y1year_test, y1year_pred)) + '.json')

In [ ]:
y1year_pred = InjuryModelXGB1year.predict(x1year_test)
print(r2_score(y1year_test, y1year_pred))
print(mean_absolute_error(y1year_test, y1year_pred))
print(mean_squared_error(y1year_test, y1year_pred))

In [ ]:
#Check sample Ousmane Dembélé, row 5999, why over 365
InjuryModelOneYeardf.sample(10)

In [ ]:
X6years_train, X6years_test, y6years_train, y6years_test = train_test_split(x6years, y6years, random_state=1, train_size = 0.8)
InjuryModelXGB6years = xgb.XGBRegressor(tree_method="approx", seed=1, enable_categorical = True)
InjuryModelXGB6years.fit(X6years_train, y6years_train, verbose = True, early_stopping_rounds = 10, eval_set = [(X6years_test, y6years_test)])

In [ ]:
y6years_pred = InjuryModelXGB6years.predict(X6years_test)
print(r2_score(y6years_test, y6years_pred))
print(mean_absolute_error(y6years_test, y6years_pred))
print(mean_squared_error(y6years_test, y6years_pred))

In [ ]:
#Shap test
graphviz6years = xgb.to_graphviz(InjuryModelXGB6years)
explainer6years = shap.Explainer(graphviz6years)
shap_values6years = explainer6years.shap_values(X6years_test)
explainer6years.plot_importance(shap_values6years, X6years_test)
shap.summary_plot(shap_values6years, X6years_test)

In [ ]:
## injury prediction for 1 year sliding window
## need to check date joined club, if change club, change values to -1, should be fixed since injury data on all history of player recorded
#drop player value column

#concatenate repeated players into same row
#add features into methods
#sliding window, rolling
#create numpy array to store r squared/mean squared of accuracy of each model with adjusted hyperparameters
#start with set seed
#5-10 seeds, find average of models
#cross validation